In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf
import json
import cv2
import sklearn
from tensorflow.keras import regularizers
from tensorflow.keras import layers
import os

In [2]:
GESTURE_TYPES = 11
LABEL_DICT = {k:i for i,k in enumerate([21, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33])}
CONNECTION_LABELS = [
    (0, 1), (1, 2), (2, 3), (3, 4),
    (5, 6), (6, 7), (7, 8),
    (9, 10), (10, 11), (11, 12),
    (13, 14), (14, 15), (15, 16),
    (17, 18), (18, 19), (19, 20),
    (0, 5), (5, 9), (9, 13), (13, 17), (0, 17)
]

In [8]:
def load_keypoint_sequences(data_path='gesture_recognition/Fall 2020/data', sequence_length=5):
    keypoints = []
    labels = []
    for subjectName in os.listdir(data_path):
        if not (subjectName.startswith("Subject") or subjectName.startswith("subject")): continue
        # subjectNum = int(re.findall(r'(\d+)', subjectName)[0])
        for sceneName in os.listdir(os.path.join(data_path, subjectName)):
            if not (sceneName.startswith("Scene") or subjectName.startswith("scene")): continue
            for groupEntry in os.scandir(os.path.join(data_path, subjectName, sceneName)):
                with open(groupEntry, 'r') as f:
                    groupData = json.load(f)
                    for gesture in groupData:
                        # print(gesture['label'], gesture['keypoints'])
                        for i in range(len(gesture['keypoints'])):
                            if not gesture['keypoints'][i]:
                                gesture['keypoints'][i] = [[np.nan, np.nan, np.nan] for _ in range(21)]
                        for start_frame in range(len(gesture['keypoints']) - sequence_length + 1):
                            keypoints.append(gesture['keypoints'][start_frame: start_frame + sequence_length])
                            labels.append(LABEL_DICT[gesture['label']])
    keypoints = np.array(keypoints)
    labels = np.array(labels)
    return keypoints, labels
keypoints, labels = load_keypoint_sequences()
print(keypoints.shape, labels.shape)

(23346, 5, 21, 3) (23346,)


In [ ]:
def generate_connection_angles_from_sequences(keypoints, keypoints_num=21, keypoints_dimensions=3):
    connections = []
    for connection in CONNECTION_LABELS:
        connections.append(keypoints[..., connection[1], :] - keypoints[..., connection[0], :])
    connections = np.stack(connections, axis = -2)
    tensor1 = connections[..., np.newaxis].repeat(keypoints_num, -1).transpose(0,1,2,4,3)
    tensor2 = connections[..., np.newaxis].repeat(keypoints_num, -1).transpose(0,1,4,2,3)
    angles = (tensor1*tensor2).sum(axis=-1)/np.linalg.norm(tensor1,axis=-1)/np.linalg.norm(tensor2,axis=-1)
    angles = angles.transpose(2,3,0,1)[np.triu_indices(21, k = 1)].transpose(1,2,0)
    return np.arccos(angles)
angles = generate_connection_angles_from_sequences(keypoints)
print(angles.shape)

In [ ]:
def generate_joint_distances_from_sequences(keypoints, keypoints_num=21, keypoints_dimensions=3):
    connections = []
    for connection in CONNECTION_LABELS:
        connections.append(keypoints[..., connection[1], :] - keypoints[..., connection[0], :])
    connections = np.stack(connections, axis = -2)
    tensor1 = connections[..., np.newaxis].repeat(keypoints_num, -1).transpose(0,1,2,4,3)
    tensor2 = connections[..., np.newaxis].repeat(keypoints_num, -1).transpose(0,1,4,2,3)
    distances = np.linalg.norm(tensor1-tensor2,axis=-1).transpose(2,3,0,1)[np.triu_indices(21, k = 1)].transpose(1,2,0)
    return distances
distances = generate_joint_distances_from_sequences(keypoints)
print(distances.shape)

In [ ]:
def visualize_keypoint_sequences(keypoints):
    for sequence in keypoints:
        for points in sequence:
            img = np.zeros((480, 640, 3))
            for point in points:
                x, y, z = point
                if np.isnan(x):
                    continue
                cv2.circle(img, (int(x), int(y)), 4, (255, 0, 0), 2)
            for connection in CONNECTION_LABELS:
                if np.isnan(points[connection[0]][0]):
                    continue
                x0, y0, z0 = points[connection[0]]
                x1, y1, z1 = points[connection[1]]
                cv2.line(img, (int(x0), int(y0)), (int(x1), int(y1)), (0, 255, 0), 2)
            cv2.imshow("Key Points", img)
            key = cv2.waitKey(1)
            if key == 27:
                cv2.destroyAllWindows()
                cv2.waitKey(1) # cannot close window on macOS without this line
                return
# visualize_keypoint_sequences(keypoints)

In [45]:
def process_sequence_features(keypoints, angles, distances):
    data_length = keypoints.shape[0]
    sequence_length = keypoints.shape[1]
    keypoints = keypoints.reshape(data_length*sequence_length, -1)
    angles = angles.reshape(data_length*sequence_length, -1)
    distances = distances.reshape(data_length*sequence_length, -1)
    features = np.concatenate((keypoints, angles, distances), -1)
    df = pd.DataFrame(features)
    df = (df-df.mean())/df.std()
    df = df.fillna(0)
    features = df.to_numpy().reshape(data_length, sequence_length, -1)
    return features
X = process_sequence_features(keypoints, angles, distances)
#normalizer = tf.keras.layers.experimental.preprocessing.Normalization()
#normalizer.adapt(X)
# X_train, X_val, y_train, y_val = train_test_split(processed_keypoints, labels, test_size=0.2, random_state=0)
print(X.shape)

(18250, 5, 483)


## RNN Classification

In [46]:
model_lstm = tf.keras.Sequential([layers.Masking() ,layers.LSTM(GESTURE_TYPES, activation=None), layers.Activation('softmax')])
model_lstm.compile(loss=tf.losses.SparseCategoricalCrossentropy(), optimizer=tf.optimizers.Adam(), metrics=['accuracy'])
history = model_lstm.fit(X, labels, epochs=20, validation_split=0.2)

Epoch 1/20
457/457 [==============================] - 3s 5ms/step - loss: 0.9810 - accuracy: 0.6862 - val_loss: 0.8193 - val_accuracy: 0.8044
Epoch 2/20
457/457 [==============================] - 2s 4ms/step - loss: 0.4230 - accuracy: 0.8736 - val_loss: 0.7648 - val_accuracy: 0.8115
Epoch 3/20
457/457 [==============================] - 2s 4ms/step - loss: 0.3609 - accuracy: 0.8897 - val_loss: 0.9026 - val_accuracy: 0.8110
Epoch 4/20
457/457 [==============================] - 2s 4ms/step - loss: 0.3389 - accuracy: 0.8955 - val_loss: 0.9116 - val_accuracy: 0.8252
Epoch 5/20
457/457 [==============================] - 2s 4ms/step - loss: 0.3046 - accuracy: 0.9104 - val_loss: 0.9243 - val_accuracy: 0.8293
Epoch 6/20
457/457 [==============================] - 2s 4ms/step - loss: 0.2909 - accuracy: 0.9149 - val_loss: 1.0533 - val_accuracy: 0.8156
Epoch 7/20
457/457 [==============================] - 2s 4ms/step - loss: 0.2750 - accuracy: 0.9170 - val_loss: 1.0856 - val_accuracy: 0.8266
Epoch 

In [ ]:
model_gru = tf.keras.Sequential([layers.Masking() ,layers.GRU(GESTURE_TYPES, activation=None), layers.Activation('softmax')])
model_gru.compile(loss=tf.losses.SparseCategoricalCrossentropy(), optimizer=tf.optimizers.Adam(), metrics=['accuracy'])
history = model_gru.fit(X, labels, epochs=20, validation_split=0.2)

GRU cells work just as well as LSTM cells, and is cheaper in theory.

In [22]:
model_bilstm = tf.keras.Sequential([layers.Masking(), tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(GESTURE_TYPES, activation=None)), tf.keras.layers.Activation('softmax')])
model_bilstm.compile(loss=tf.losses.SparseCategoricalCrossentropy(), optimizer=tf.optimizers.Adam(), metrics=['accuracy'])
history = model_bilstm.fit(X, labels, epochs=20, validation_split=0.2)

Train on 18676 samples, validate on 4670 samples
Epoch 1/20
18676/18676 [==============================] - 18s 946us/sample - loss: 0.6432 - accuracy: 0.8113 - val_loss: 0.6360 - val_accuracy: 0.8109
Epoch 2/20
18676/18676 [==============================] - 12s 664us/sample - loss: 0.3715 - accuracy: 0.8841 - val_loss: 0.6390 - val_accuracy: 0.8193
Epoch 3/20
18676/18676 [==============================] - 13s 682us/sample - loss: 0.3176 - accuracy: 0.8992 - val_loss: 0.6400 - val_accuracy: 0.8188
Epoch 4/20
18676/18676 [==============================] - 13s 691us/sample - loss: 0.2875 - accuracy: 0.9074 - val_loss: 0.6890 - val_accuracy: 0.8135
Epoch 5/20
18676/18676 [==============================] - 13s 699us/sample - loss: 0.2707 - accuracy: 0.9118 - val_loss: 0.6987 - val_accuracy: 0.8218
Epoch 6/20
18676/18676 [==============================] - 14s 737us/sample - loss: 0.2518 - accuracy: 0.9185 - val_loss: 0.6891 - val_accuracy: 0.8263
Epoch 7/20
18676/18676 [=====================

Making LSTM bidirectional does not improve the accuracy by a lot.

In [24]:
model_rnn = tf.keras.Sequential([layers.Masking() ,tf.keras.layers.SimpleRNN(GESTURE_TYPES, activation=None), tf.keras.layers.Activation('softmax')])
model_rnn.compile(loss=tf.losses.SparseCategoricalCrossentropy(), optimizer=tf.optimizers.Adam(), metrics=['accuracy'])
history = model_rnn.fit(X, labels, epochs=20, validation_split=0.2)

Train on 18676 samples, validate on 4670 samples
Epoch 1/20
18676/18676 [==============================] - 12s 623us/sample - loss: 0.9693 - accuracy: 0.7050 - val_loss: 0.8205 - val_accuracy: 0.7458
Epoch 2/20
18676/18676 [==============================] - 7s 381us/sample - loss: 0.5730 - accuracy: 0.8219 - val_loss: 0.7488 - val_accuracy: 0.7747
Epoch 3/20
18676/18676 [==============================] - 7s 349us/sample - loss: 0.4934 - accuracy: 0.8461 - val_loss: 0.7004 - val_accuracy: 0.7895
Epoch 4/20
18676/18676 [==============================] - 6s 332us/sample - loss: 0.4533 - accuracy: 0.8579 - val_loss: 0.7124 - val_accuracy: 0.7979
Epoch 5/20
18676/18676 [==============================] - 7s 360us/sample - loss: 0.4359 - accuracy: 0.8646 - val_loss: 0.7233 - val_accuracy: 0.7989
Epoch 6/20
18676/18676 [==============================] - 7s 350us/sample - loss: 0.4157 - accuracy: 0.8695 - val_loss: 0.6869 - val_accuracy: 0.8032
Epoch 7/20
18676/18676 [==========================

Simple RNN gives a slightly worse performance.

In [25]:
model_lstm4 = tf.keras.Sequential([layers.Masking(), tf.keras.layers.LSTM(128), layers.Dense(GESTURE_TYPES), layers.Activation('softmax')])
model_lstm4.compile(loss=tf.losses.SparseCategoricalCrossentropy(), optimizer=tf.optimizers.Adam(), metrics=['accuracy'])
history = model_lstm4.fit(X, labels, epochs=20, validation_split=0.2)

Train on 18676 samples, validate on 4670 samples
Epoch 1/20
18676/18676 [==============================] - 25s 1ms/sample - loss: 0.4140 - accuracy: 0.8728 - val_loss: 0.5526 - val_accuracy: 0.8223
Epoch 2/20
18676/18676 [==============================] - 20s 1ms/sample - loss: 0.2272 - accuracy: 0.9288 - val_loss: 0.5408 - val_accuracy: 0.8332
Epoch 3/20
18676/18676 [==============================] - 21s 1ms/sample - loss: 0.1595 - accuracy: 0.9516 - val_loss: 0.5519 - val_accuracy: 0.8379
Epoch 4/20
18676/18676 [==============================] - 20s 1ms/sample - loss: 0.1228 - accuracy: 0.9641 - val_loss: 0.5707 - val_accuracy: 0.8281
Epoch 5/20
18676/18676 [==============================] - 20s 1ms/sample - loss: 0.0874 - accuracy: 0.9768 - val_loss: 0.5883 - val_accuracy: 0.8362
Epoch 6/20
18676/18676 [==============================] - 21s 1ms/sample - loss: 0.0699 - accuracy: 0.9823 - val_loss: 0.6610 - val_accuracy: 0.8317
Epoch 7/20
18676/18676 [==============================] -

Increasing dense layer depth may improve performance.

In [12]:
model_lstm5 = tf.keras.Sequential([layers.Masking(), tf.keras.layers.LSTM(128,return_sequences=True),tf.keras.layers.LSTM(128), layers.Dense(GESTURE_TYPES), layers.Activation('softmax')])
model_lstm5.compile(loss=tf.losses.SparseCategoricalCrossentropy(), optimizer=tf.optimizers.Adam(), metrics=['accuracy'])
history = model_lstm5.fit(X, labels, epochs=20, validation_split=0.2)

Train on 18676 samples, validate on 4670 samples
Epoch 1/20
18676/18676 [==============================] - 58s 3ms/sample - loss: 0.4245 - accuracy: 0.8673 - val_loss: 0.6317 - val_accuracy: 0.8090
Epoch 2/20
18676/18676 [==============================] - 41s 2ms/sample - loss: 0.2202 - accuracy: 0.9286 - val_loss: 0.5720 - val_accuracy: 0.8313
Epoch 3/20
18676/18676 [==============================] - 39s 2ms/sample - loss: 0.1491 - accuracy: 0.9512 - val_loss: 0.7063 - val_accuracy: 0.8139
Epoch 4/20
18676/18676 [==============================] - 40s 2ms/sample - loss: 0.1199 - accuracy: 0.9620 - val_loss: 0.7088 - val_accuracy: 0.8180
Epoch 5/20
18676/18676 [==============================] - 37s 2ms/sample - loss: 0.0935 - accuracy: 0.9701 - val_loss: 0.7491 - val_accuracy: 0.8278
Epoch 6/20
18676/18676 [==============================] - 42s 2ms/sample - loss: 0.0701 - accuracy: 0.9780 - val_loss: 0.7608 - val_accuracy: 0.8201
Epoch 7/20
18676/18676 [==============================] -

Adding another layer of LSTM does not improve performance.

## Convolutional + Recurrent
[Núñez et al. - Convolutional Neural Networks and Long Short-Term Memory for skeleton-based human activity and hand gesture recognition](https://www.sciencedirect.com/science/article/pii/S0031320317304405)

In [16]:
model_cnnlstm2 = tf.keras.Sequential([layers.Masking(), layers.Conv1D(20,3,activation='relu'),layers.Conv1D(20,3,activation='relu'), layers.LSTM(128, return_sequences=True), layers.Dense(GESTURE_TYPES, activation='softmax')])
model_cnnlstm2.compile(loss=tf.losses.SparseCategoricalCrossentropy(), optimizer=tf.optimizers.Adam(), metrics=['accuracy'])
history = model_cnnlstm2.fit(X, labels, epochs=20, validation_split=0.2)

Train on 18676 samples, validate on 4670 samples
Epoch 1/20
18676/18676 [==============================] - 13s 688us/sample - loss: 0.6499 - accuracy: 0.7915 - val_loss: 0.6435 - val_accuracy: 0.7953
Epoch 2/20
18676/18676 [==============================] - 9s 486us/sample - loss: 0.3910 - accuracy: 0.8702 - val_loss: 0.6158 - val_accuracy: 0.8045
Epoch 3/20
18676/18676 [==============================] - 9s 497us/sample - loss: 0.3348 - accuracy: 0.8911 - val_loss: 0.6311 - val_accuracy: 0.8026
Epoch 4/20
18676/18676 [==============================] - 10s 560us/sample - loss: 0.3003 - accuracy: 0.8982 - val_loss: 0.6754 - val_accuracy: 0.7949
Epoch 5/20
18676/18676 [==============================] - 10s 525us/sample - loss: 0.2730 - accuracy: 0.9090 - val_loss: 0.6451 - val_accuracy: 0.8122
Epoch 6/20
18676/18676 [==============================] - 10s 518us/sample - loss: 0.2506 - accuracy: 0.9143 - val_loss: 0.7023 - val_accuracy: 0.8009
Epoch 7/20
18676/18676 [=======================